In [8]:
from concurrent import futures

from selenium import webdriver
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities 
from selenium.webdriver.support.ui import WebDriverWait as wait 
from selenium.webdriver.common.by import By 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

from selenium.common.exceptions import TimeoutException

from time import sleep
import time

import os

#버전 관리용 라이브러리
import chromedriver_autoinstaller

import subprocess
import shutil

import pyperclip

import datetime
import urllib.request

import pandas as pd

import random

In [9]:
user_info = pd.read_csv('./info.csv')
#몇 개의 계정을 돌릴건지 확인 -> 쓰레드 갯수 때문에 필요함
user_num = len(user_info)
user_info

,DIR_NUM,ID,PW,PROXY,LINK,SIZE
0,0,ksh415622@naver.com,Testtest123*,209.61.207.249:64668:run:rKeLShPi,https://www.nike.com/kr/launch/t/men/fw/nike-s...,250
1,1,csy3984,tndyd674,209.61.207.133:58758:run:rKeLShPi,https://www.nike.com/kr/launch/t/men/fw/nike-s...,240
2,2,yhotsky,tndyd674!,209.61.207.134:58299:run:rKeLShPi,https://www.nike.com/kr/launch/t/men/fw/nike-s...,260
3,3,tjrgh0914@nate.com,Depuis1979@,209.61.207.131:59596:run:rKeLShPi,https://www.nike.com/kr/launch/t/men/fw/nike-s...,250
4,4,kenmarkess@gmail.com,Ken710039!,209.61.207.132:64219:run:rKeLShPi,https://www.nike.com/kr/launch/t/men/fw/nike-s...,240


In [10]:

def check_time(hour, minute,meridiem):
    # if meridiem == 'PM':
    #     hour = str(int(hour) + 12)
    url = 'https://www.naver.com/'
    date = urllib.request.urlopen(url).headers['Date'][5:-4]
    nav_hour, nav_min, nav_sec =  date[12:14], date[15:17], date[18:]
    nav_hour = str((int(nav_hour)+ 9)%12)

    nav_hour = int(nav_hour)
    nav_min = int(nav_min)

    hour = int(hour)
    minute = int(minute)
    
    print(nav_hour, ", " , nav_min, ", ", nav_sec)
    if (nav_hour == hour ) and (nav_min == minute):
        return True
    else: 
        return False
    
# input_hour = input("시간을 입력해주세요: ") 
# input_min = input("분을 입력해주세요: ")
# input_med = input("AM or PM: ")

input_hour = "10"
input_min = "00"
input_med = "PM"

In [11]:
def check_logged_in(Chrome_driver,user_ID,user_PW, user_num):
    sleep(5)
    sleep(user_num)
    
    try:
        action = ActionChains(Chrome_driver)
        login_btn = WebDriverWait(Chrome_driver,5).until(EC.presence_of_element_located((By.XPATH, '/html/body/header/div/div[1]/div/div/ul/li[3]/a')))
#         login_btn = driver.find_element(By.XPATH,'/html/body/header/div/div[1]/div/div/ul/li[3]/a')
        action.move_to_element(login_btn).click().perform()
    except TimeoutException:
        print(str(user_num) + '번 째 아이디는 이미 로그인 되어 있습니다')
        return
    
    sleep(1)
    pyperclip.copy(user_ID)
    ID_input = WebDriverWait(Chrome_driver,5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="j_username"]')))
    for i in range(50):
        sleep(0.01)
        ID_input.send_keys(Keys.BACKSPACE)
    ID_input.send_keys(Keys.CONTROL + 'v')

    sleep(1)
    pyperclip.copy(user_PW)
    PW_input = WebDriverWait(Chrome_driver,5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="j_password"]')))
    for i in range(50):
        sleep(0.01)
        PW_input.send_keys(Keys.BACKSPACE)
    PW_input.send_keys(Keys.CONTROL + 'v')


    action = ActionChains(Chrome_driver)
    submit_btn = WebDriverWait(Chrome_driver,5).until(EC.presence_of_element_located((By.XPATH, '//*[@id="common-modal"]/div/div/div/div[2]/div/div[2]/div/button')))
    #무작위로 딜레이 주기
    sleep(user_num)
    action.move_to_element(submit_btn).click().perform()

In [12]:

def choose_address(Chrome_driver):
    action = ActionChains(Chrome_driver)
    sleep(1)
    # default_addr = WebDriverWait(Chrome_driver,3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="shipping_info"]/div[1]/ul/li[3]/div/span/label/i')))
    # while 1:
    #     action.move_to_element(default_addr).click().perform()
    #     sleep(0.2)
    #     if(Chrome_driver.page_source.find("input-checkbox checked") != -1):
    #         break
    #     else:
    #         sleep(1)
    #         if(Chrome_driver.page_source.find("input-checkbox checked") != -1):
    #             break
    
    next_stage = WebDriverWait(Chrome_driver,3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btn-next"]')))
    action.move_to_element(next_stage).click().perform()

def choose_payment(Chrome_driver):
    while 1:
        action = ActionChains(Chrome_driver)
        sleep(0.7)
        payment = WebDriverWait(Chrome_driver,2).until(EC.presence_of_element_located((By.XPATH, '//*[@id="payment-review"]/div[1]/ul/li[1]/div/div[1]/h6/img')))
        action.move_to_element(payment).click().perform()
        sleep(0.05)
        terms_of_conditions = WebDriverWait(Chrome_driver,2).until(EC.presence_of_element_located((By.XPATH,'//*[@id="payment-review"]/div[1]/ul/li[2]/form/div/span/label/i')))
        action.move_to_element(terms_of_conditions).click().perform()
        sleep(1.5)
        complete_purchase = WebDriverWait(Chrome_driver,2).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="complete_checkout"]/button')))
        action.move_to_element(complete_purchase).click().perform()
        
        


In [15]:

def goto_page(Chrome_driver, link, size, get_size_mode):
    Chrome_driver.get(link)
    sleep(1)
    #url에 "launch"가 없을 때 (스텔스(?))
    if(link.find('launch') == -1):
        #----------------------예외 처리--------------------
        for i in range(5): 
            if(Chrome_driver.page_source.find("사이즈 선택") == -1):
                Chrome_driver.get(link)
                sleep(3)
        #----------------------예외 처리--------------------
        action = ActionChains(Chrome_driver)

        while 1:
            sleep(1.5)
            if(get_size_mode == "select_size"):
                size_element = WebDriverWait(Chrome_driver, 3).until(EC.element_to_be_clickable((By.XPATH,\
                    '/html/body/section/section/section/article/article[2]/div/div[4]/div/div[2]/form/div[2]/div[2]/div[1]/div/span[*]/label[text()=' + size  + ']')))
                sleep(1.3)
                action.move_to_element(size_element).click().perform()

                sleep(0.1)
                purchase_btn =  WebDriverWait(Chrome_driver, 3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btn-buy"]/span')))
                action.move_to_element(purchase_btn).click().perform()
                sleep(0.2)
            elif(get_size_mode == "random_size"):
                size_elements = Chrome_driver.find_elements(By.XPATH, \
                    '/html/body/section/section/section/article/article[2]/div/div[4]/div/div[2]/form/div[2]/div[2]/div[1]/div/span[not(@disabled="disabled")]')
                random_size = random.randint(0,len(size_elements))
                sleep(0.1)
                size_element = size_elements[random_size]
                action.move_to_element(size_element).click().perform()
                purchase_btn =  WebDriverWait(Chrome_driver, 3).until(EC.element_to_be_clickable((By.XPATH,  '//*[@id="btn-buy"]/span')))
                action.move_to_element(purchase_btn).click().perform()
                sleep(0.2)
            if(Chrome_driver.page_source.find("사이즈를 선택해 주세요") == -1):
                break
            else:
                get_size_mode="random_size"

    #url에 "launch"가 있을 때 (스니커즈)
    else:
        #----------------------아직 발매가 시작 안됐을 때--------------------
        for i in range(5): 
            if(Chrome_driver.page_source.find("사이즈 선택") == -1): 
                Chrome_driver.get(link)
                sleep(3)
        #----------------------아직 발매가 시작 안됐을 때--------------------
        action = ActionChains(Chrome_driver)
        while 1:
            sleep(1.5)
            if(get_size_mode == "select_size"):
                size_list = WebDriverWait(Chrome_driver, 3).until(EC.presence_of_element_located((By.XPATH, \
                        '/html/body/div[1]/div/div[1]/div[2]/div[1]/section/div[2]/aside/div[2]/div/div/div/div/form/div/div[1]/a/span')))
                action.move_to_element(size_list).click().perform()
                
                sleep(0.5)
                size_element = WebDriverWait(Chrome_driver, 3).until(EC.presence_of_element_located((By.XPATH,\
                        '/html/body/div[1]/div/div[1]/div[2]/div[1]/section/div[2]/aside/div[2]/div/div/div/div/form/div/div[1]/ul/li[*]/a/span[text()=' + size  + ']')))
                action.move_to_element(size_element).click().perform()
                
                #구매 버튼 누르기 전에 아무 곳 클릭 (여기서는 상품 이름 명 클릭)
                sleep(0.1)
                try: 
                    temp_element = WebDriverWait(Chrome_driver, 3).until(EC.presence_of_element_located((By.XPATH,\
                        '/html/body/div[1]/div/div[1]/div[2]/div[1]/section/div[2]/aside/div[1]/div/h1')))
                except:
                    temp_element = WebDriverWait(Chrome_driver, 3).until(EC.presence_of_element_located((By.XPATH,\
                        '/html/body/div[1]/div/div[1]/div[2]/div[1]/section/div[2]/aside/div[1]/div[1]/div/h1')))

                action.move_to_element(temp_element).click().perform()
                

                sleep(0.1)
                purchase_btn =  WebDriverWait(Chrome_driver, 3).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btn-buy"]/span')))
                action.move_to_element(purchase_btn).click().perform()
            
            #random으로 구매할 수 있는 사이즈를 선택
            elif(get_size_mode == "random_size"):
                size_list = WebDriverWait(Chrome_driver, 3).until(EC.presence_of_element_located((By.XPATH, \
                        '/html/body/div[1]/div/div[1]/div[2]/div[1]/section/div[2]/aside/div[2]/div/div/div/div/form/div/div[1]/a/span')))
                action.move_to_element(size_list).click().perform()
                
                sleep(0.7)
                size_elements = Chrome_driver.find_elements(By.XPATH, \
                        '/html/body/div[1]/div/div[1]/div[2]/div[1]/section/div[2]/aside/div[2]/div/div/div/div/form/div/div[1]/ul/li[@class="list"]')
                random_size = random.randint(0,len(size_elements))
                print("(launch)random_size: ", random_size)
                sleep(0.1)
                size_element = size_elements[random_size]
                action.move_to_element(size_element).click().perform()
                
                
                sleep(0.1)
                purchase_btn =  WebDriverWait(Chrome_driver, 3).until(EC.element_to_be_clickable((By.XPATH,  '//*[@id="btn-buy"]/span')))
                action.move_to_element(purchase_btn).click().perform()
            
            if(Chrome_driver.page_source.find("사이즈를 선택해 주세요") == -1):
                break
            else:
                get_size_mode = "random_size"
        

In [16]:

def init(user_num):
    #--------------------------아이디 패스워드 프록시 설정--------------------------
    ID = str(user_info[user_info['DIR_NUM']==user_num]['ID'].values[0])
    PW = str(user_info[user_info['DIR_NUM']==user_num]['PW'].values[0])
    PROXY = str(user_info[user_info['DIR_NUM']==user_num]['PROXY'].values[0])
    LINK = str(user_info[user_info['DIR_NUM']==user_num]['LINK'].values[0])
    SIZE = str(user_info[user_info['DIR_NUM']==user_num]['SIZE'].values[0])
    #--------------------------아이디 패스워드 프록시 설정--------------------------

    #------------크롬 버전 확인 및 각 사용자별 exe파일 경로 및 쿠키 경로------------
    chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
    chrome_exe_path = "C:\Program Files\Google\Chrome\Application\chrome.exe"
#     chrome_exe_path = str(os.path.abspath(os.getcwd()))
#     chrome_exe_path = chrome_exe_path.replace('\개발용dir','')
#     chrome_exe_path = chrome_exe_path + "\\" +str(user_num)  + '\Chrome\Application\chrome.exe'

    chrome_cookie_path = str(os.path.abspath(os.getcwd()))
    chrome_cookie_path = chrome_cookie_path.replace('\개발용dir','')
    chrome_cookie_path = chrome_cookie_path + "\\" +str(user_num) + '\Chrome_cookie'

    chrome_driver_path = str(os.path.abspath(os.getcwd()))
    chrome_driver_path = chrome_driver_path.replace('\개발용dir','')
    chrome_driver_path = chrome_driver_path + "\\" +str(user_num) + '\\' + chrome_ver + '\chromedriver.exe'
    #------------크롬 버전 확인 및 각 사용자별 exe파일 경로 및 쿠키 경로------------
    
    port_num = 9222 + user_num
    
    #------------------------자동 제어모드 우회------------------------------
    subprocess.Popen(r'{exe} --remote-debugging-port={port_num} --user-data-dir="{cookie}"'.format(exe=chrome_exe_path, cookie=chrome_cookie_path, port_num=str(port_num)))
#     #------------------------자동 제어모드 우회------------------------------
    
    
#     #---------------------------프록시 서버 설정------------------------------
    webdriver.DesiredCapabilities.CHROME['proxy'] = {
        "httpProxy": PROXY,
        "ftpProxy": PROXY,
        "sslProxy": PROXY,
        "proxyType": "MANUAL"
    }

    #페이지가 다 로드 되기 전에 다음 코드 실행할 수 있도록 설정
    caps = DesiredCapabilities().CHROME 
    caps["pageLoadStrategy"] = "none"

    #디버거 모드로 설정(이렇게 안하면 로그인이 다 막힘)
    option = Options()
    option.add_argument("--start-maximized")
    option.add_experimental_option("debuggerAddress", f"127.0.0.1:{port_num}")
    option.add_argument('--blink-settings=imagesEnabled=false')
    option.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36")
    option.add_argument(f'--user-data-dir={chrome_cookie_path}')

    #크롬 드라이버 설정
    try:
        driver = webdriver.Chrome(executable_path=f'{chrome_driver_path}',desired_capabilities=caps, options=option)
    except:
        chromedriver_autoinstaller.install(True)
        driver = webdriver.Chrome(executable_path=f'{chrome_driver_path}',desired_capabilities=caps, options=option)
    
    driver.implicitly_wait(3)  
    driver.get("https://www.nike.com/kr/ko_kr/")
    
    try:
        driver.maximize_window()
    except:
        pass
    
    sleep(4)
    check_logged_in(driver,ID, PW, user_num)
    sleep(5)
    

#     if(input_hour 



    #임시
#     sleep(1)
#     #임시

    job_condition = "choose_size"
    for i in range(5):
        if(job_condition=="choose_size"):
            print("111")
            get_size_mode = "select_size"
            job_condition = first_step(driver, LINK, SIZE, get_size_mode, job_condition)

        if(job_condition=="choose address"):
            print("222")
            job_condition = second_step(driver,job_condition)

        if(job_condition=="choose_payment"):
            print("333")
            choose_payment(driver)
            #주문 생성 오류 감지
            sleep(3)
            #1. 오류 없이 결제 창이 잘 떴을 경우
            if(driver.page_source.find("오류가") == -1):
                break
            else:
                action = ActionChains(driver)
                submit_btn =  WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[17]/div/div/div[2]/button')))
                action.move_to_element(submit_btn).click().perform()
                
    
def first_step(driver, LINK, SIZE, get_size_mode, job_condition="choose_size"):
    #사이즈 고른 후 주소 선택 화면을 넘어가지 않을 시 5번 시도 (더 많이 하면 no access 뜸)
    for i in range(5):
        goto_page(driver,LINK,SIZE,get_size_mode)
        sleep(2)
        #사이즈 선택 및 구매버튼 클릭 완료 후(여기 부분의 예외처리는 goto_page 안에 구현되어 있음)
        #배송지 선택 창으로 넘어가는지 확인
        #no-access 혹은 사이즈 선택 창에서 성공적으로 실행되지 않았다면 random_size 모드로 사이즈 선택 총 5번 실행
        if(driver.current_url.find('checkout') == -1):
            get_size_mode = 'random_size'
        else:
            job_condition = 'choose address'
            return job_condition

def second_step(driver,job_condition="choose address"):
    for i in range(5):
        choose_address(driver)
        sleep(2)
        #배송지 선택 화면으로 넘어온 후 payment선택 화면으로 정상적으로 이동하지 못했을 시
        #1. no-access이면 job_condition을 사이즈 선택으로 다시 돌아가게
        #2. 다른 오류로 payment 창으로 넘어가지 못했을 경우 새로고침 후 다시 배송지 선택
        #3. payment로 성공적으로 넘어가면 다음 스탭 시도
        print("here")
        if(driver.current_url.find('checkout') == -1):
            if(driver.current_url.find('error/no-access') != -1):
                job_condition = 'choose_size'
                return job_condition
            else:
                continue
        else:
            # if(driver.page_source.find('실시간계좌이체') != 1):
            job_condition = 'choose_payment'
            return job_condition







with futures.ThreadPoolExecutor(max_workers=20) as executor: 
    future_test_results = [ executor.submit(init, i) for i in range(5) ] # running same test 6 times, using test number as url
    for future_test_result in future_test_results: 
        try:        
            test_result = future_test_result.result(timeout=None) # can use `timeout` to wait max seconds for each thread               
            #... do something with the test_result
        except: # can give a exception in some thread, but 
            print('thread generated an exception: {:0}'.format(Exception))

C:\Users\Administrator\AppData\Local\Temp\2\ipykernel_15480\242946149.py:55: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=f'{chrome_driver_path}',desired_capabilities=caps, options=option)


0번 째 아이디는 이미 로그인 되어 있습니다
1번 째 아이디는 이미 로그인 되어 있습니다
2번 째 아이디는 이미 로그인 되어 있습니다
3번 째 아이디는 이미 로그인 되어 있습니다
4번 째 아이디는 이미 로그인 되어 있습니다
111
111
111
111
111
222
222
222
here
333
222
here
333
here
222
333
here
333
here
333


TypeError: unsupported format string passed to type.__format__